# FastSMC example notebook

This notebook demonstrates how to use the FastSMC python bindings, with optional configuration of parameters.

Please make sure you have installed the python bindings by following the instructions in `../README.md` before attempting to run this notebook.

The example dataset was simulated using the setup described in the paper, corresponding to SNP data for 150 diploid individuals and a chromosomal region of 30 Mb, with recombination rate from chromosome 2 and under a European demographic model (see https://www.biorxiv.org/content/10.1101/2020.04.20.029819v1 for more details).

1) Import `asmc` which is installed with the Python bindings

In [ ]:
from asmc.asmc import *

import pathlib
import tempfile

data_dir = pathlib.Path('.').resolve().parent / 'ASMC_data'

2) Specify paths for input (example provided with this repository) and output

In [ ]:
input_files_root = str(data_dir / 'examples' / 'fastsmc' / 'example')
dq_file = str(data_dir / 'decoding_quantities' / '30-100-2000_CEU.decodingQuantities.gz')
output_files_root = tempfile.TemporaryDirectory().name

3) Set the required parameters by creating a DecodingParams object. When creating parameters in this manner, you should run `validateParamsFastSMC()` to ensure you have not selected incompatible options. In this notebook, data is saved in a binary format (`params.BIN_OUT = True`). This is more space efficient and should be used for a large analysis.

In [ ]:
params = DecodingParams()
params.decodingQuantFile = dq_file
params.inFileRoot = input_files_root
params.outFileRoot = output_files_root
params.decodingModeString = 'array'
params.usingCSFS = True
params.batchSize = 32
params.recallThreshold = 3
params.min_m = 1.5
params.hashing = True
params.FastSMC = True
params.BIN_OUT = True
params.outputIbdSegmentLength = True
params.time = 50
params.noConditionalAgeEstimates = True
params.doPerPairMAP = True
params.doPerPairPosteriorMean = True

assert params.validateParamsFastSMC()

4) Create the Python FastSMC object and run it. This should only take a few seconds.

In [ ]:
fast_smc = FastSMC(params)
fast_smc.run()

5) Read data from the binary file using the `BinaryDataReader` class. Because the data file is assumed to be very large, reading the file should be done line-by-line as the file will not necessarily fit in memory. We then filter to remove IBD segments with low IBD score, and bin values for the histograms we want to plot. Note that we demonstrate here how to read the output line-by-line as for a large analysis the data may not all fit in memory.

In [ ]:
%config InlineBackend.figure_formats = ['svg']

import gzip
import math
import numpy as np
import matplotlib.pyplot as plt

vals_MAP = np.linspace(0, 100, num=10)
bins_MAP = np.zeros((10,), dtype=int)

vals_segLen = np.linspace(0., 15., num=10)
bins_segLen = np.zeros((10,), dtype=int)

binary_data_reader = BinaryDataReader(output_files_root + '.1.1.FastSMC.bibd.gz')

while binary_data_reader.moreLinesInFile():
    line = binary_data_reader.getNextLine()
    
    if line.ibdScore > 0.1:
        
        if line.mapEst < 100.:
            bins_MAP[math.floor((line.mapEst / 10.))] += 1

        if line.lengthInCentimorgans < 15.:
            bins_segLen[math.floor(line.lengthInCentimorgans / 1.5)] += 1

6) Visualise data: here we simply plot the pre-binned data for MAP age estimates and for IBD segment length

In [ ]:
plt.xlabel("MAP age estimate (in generations)")
plt.hist(vals_MAP, weights=bins_MAP)
plt.gca().set_yscale('linear')
plt.grid(b=True, which='major', axis='both')

In [ ]:
plt.xlabel("IBD segments length (in cM)")
plt.hist(vals_segLen, weights=bins_segLen)
plt.gca().set_yscale('log')
plt.grid(b=True, which='major', axis='both')